# Lógica Computacional TP1
## Grupo 2

### Membros:

Bruno Cunha Silva - A91649

Gonçalo Duarte Gomes Rodrigues - A91641

Tiago Rafael Fernandes Guimarães - A91689
$$$$

### Implementação

Começamos por importar a biblioteca de programação linear do OR-Tools e a biblioteca random.



Neste exercicío começamos por criar o solver, tipo SCIP, do nosso problema ao qual damos o nome de $schedule$.

In [1]:
from ortools.linear_solver import pywraplp
import random

schedule = pywraplp.Solver.CreateSolver('SCIP')
P,D,T,S,C = 4,5,8,6,10

Em seguida, declaramos: 
* a matriz de alocação, $X$, com as seguintes variavéis indexadas:
    *   P, para representar os projetos de 1..$P$
    *   D, para representar os dias de 1..$D$
    *   T, para representar os tempos de 1..$T$
    *   S, para representar as salas de 1..$S$
    *   C, para representar os colaboradores de 1..$C$
* o conjunto que deve guardar os colaboradores, $collaborators$;
* a lista dos lideres, $L$;
* a lista das horas de projeto, $H$.

Definimos, ainda a função "Collab" que constrói o conjunto de colaboradores e para cada projeto escolhe um líder aleatório e um numero de reuniões entre 1 e h.

In [2]:
x = {}

for p in range(P):
    x[p] = {}
    for d in range(D):
        x[p][d] = {}
        for t in range(T):
            x[p][d][t] = {}
            for s in range(S):
                x[p][d][t][s] = schedule.BoolVar(f'X[{p}[{d}][{t}][{s}]')
                
def X(p,d,t,s):
    return x[p][d][t][s]

collaborators = {}

L = [] #liders
H = [] #nReunioes de projeto

def Collab(C,h):
    collaborators.clear()
    for p in range(P):
        H.append(random.randint(1,h))
        L.append(random.randint(0,C-1))
        for c in range(C):
            for d in range(D):
                for t in range(T):
                    collaborators[p,c,d,t] = True

Collab(C,4)

Passamos agora à modelação das restrições e à sua introdução no solver.

### Restrições:


1. Para cada dia $(D)\,$, tempo $(T)\,$ e sala $(S)\,$ existe apenas um projeto $(P)\,$.

$$\forall_d \cdot \forall_t \cdot \forall_s \cdot \quad \sum_{p<P} \space x_{p,d,t,s} \leq 1$$

In [3]:
for d in range(D):
    for t in range(T):
        for s in range(S):
            schedule.Add(sum([X(p,d,t,s) for p in range(P)]) <= 1)

2. Para qualquer projeto $(P)\,$, dia $(D)\,$ e tempo $(T)\,$, cada reunião só poderá ser numa sala.

$$\forall_p \cdot \forall_d \cdot \forall_t \cdot \quad \sum_{s<S} \space x_{p,d,t,s} \leq 1$$


In [4]:
for p in range(P):
    for d in range(D):
        for t in range(T):
            schedule.Add(sum([X(p,d,t,s) for s in range(S)]) <= 1)

3. A seguinte restrição vem da existencia de um $Lider$ por projeto $(P)\,$ que é obrigado a estar presente em todas as reuniões. 

$$
\forall p \cdot \forall d \cdot \forall t \cdot \forall s \cdot \space x_{p,d,t,s} \leq collaborators_{p,L_p,d,t}
$$

In [5]:
for p in range(P):
    for d in range(D):
        for t in range(T):
            for s in range(S):
                schedule.Add(X(p,d,t,s) <= collaborators[p,L[p],d,t])

4. Sendo que cada projeto realiza um dado número de reuniões semanais, temos então a restrição de que para qualquer projeto, o número total de reuniões de cada projeto será igual a $H_p$(um array que terá essa informação).

$$\forall_p \cdot \sum_{d<D}  \sum_{t<T} \sum_{s<S} \space x_{p,d,t,s} == H_p$$


In [6]:
for p in range(P):
    schedule.Add(sum([X(p,d,t,s) for d in range(D) for t in range(T) for s in range(S)]) == H[p])

5. Colaboradores podem ou não estar presentes na reunião, no entanto a reunião só se realiza num mínimo "quorum" sendo este 50% dos colaboradores totais.

$$
\forall p \cdot \forall d \cdot \forall t \cdot \forall s \cdot x_{p,d,t,s} \leq (\dfrac {\sum_{c} collaborators_{p,c,d,t}}{C})
$$

In [7]:
for p in range(P):
    for d in range(D):
        for t in range(T):
            for s in range(S):
                schedule.Add(X(p,d,t,s) <= (sum([collaborators[p,c,d,t] for c in range(C)])/C))

6. Para maximizar o número de reuniões efetivamente realizadas, maximiza-se a soma do numero de reuniões semanais portanto $H$.

In [8]:
schedule.Maximize(sum(H))

In [9]:
status = schedule.Solve()

if (status == pywraplp.Solver.OPTIMAL):
    for p in range(P):
        print(F'#Lista de reuniões do projeto {p+1}:')
        for d in range(D):
            for t in range(T):
                for s in range(S):
                    if x[p][d][t][s].solution_value() != 0:
                        print(F'Dia: {d+1}, Horas: {t+8}, Sala: {s+1}')

#Lista de reuniões do projeto 1:
Dia: 4, Horas: 14, Sala: 1
Dia: 4, Horas: 15, Sala: 1
#Lista de reuniões do projeto 2:
Dia: 5, Horas: 8, Sala: 1
Dia: 5, Horas: 9, Sala: 1
#Lista de reuniões do projeto 3:
Dia: 5, Horas: 10, Sala: 1
Dia: 5, Horas: 11, Sala: 1
Dia: 5, Horas: 12, Sala: 1
#Lista de reuniões do projeto 4:
Dia: 5, Horas: 13, Sala: 1
Dia: 5, Horas: 14, Sala: 1
Dia: 5, Horas: 15, Sala: 1
